In [1]:
import pandas as pd
import json

In [2]:
df = pd.DataFrame(columns=['Play','Act','Scene','Character','Line'])

In [3]:
data = None
with open('data/shakespeare_acts_scenes.json') as f:
    data = json.load(f)

In [4]:
for play_name,play_content in data.items():
    for acts,scenes in play_content.items():
        for scene,lines in scenes.items():
            for line in lines:
                if len(line)>1:
                    df = df.append({'Play':play_name.title(), 'Act':acts, 'Scene':scene, 'Character':line[0].title(), 'Line':line[1]}, ignore_index=True)

In [5]:
df['Line_chars'] = df.apply(lambda x: len(x['Line']), axis=1)

In [6]:
df.drop(df[df['Line_chars']==1].index, inplace=True)

In [9]:
import spacy
import en_core_web_lg

from spacy.tokenizer import Tokenizer

In [10]:
nlp = en_core_web_lg.load()
tokenizer = Tokenizer(nlp.vocab)

In [11]:
df['Line_spacy'] = df.apply(lambda x: (len(tokenizer(x['Line'])),tokenizer(x['Line'])), axis=1)
df[['Line_spacy_len','Line_spacy']] = df['Line_spacy'].to_list()

/home/bryan/Documents/Code/python-environments/SHAKESPEAREAN_NETWORKS/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [12]:
df.head()

,Play,Act,Scene,Character,Line,Line_chars,Line_spacy,Line_spacy_len
0,Romeo,ACT I,SCENE I,Sampson,"Gregory, on my word, we’ll not carry coals.",43,"(Gregory,, on, my, word,, we’ll, not, carry, c...",8
1,Romeo,ACT I,SCENE I,Gregory,"No, for then we should be colliers.",35,"(No,, for, then, we, should, be, colliers.)",7
2,Romeo,ACT I,SCENE I,Sampson,"I mean, if we be in choler, we’ll draw.",39,"(I, mean,, if, we, be, in, choler,, we’ll, draw.)",9
3,Romeo,ACT I,SCENE I,Gregory,"Ay, while you live, draw your neck out o’ the ...",53,"(Ay,, while, you, live,, draw, your, neck, out...",11
4,Romeo,ACT I,SCENE I,Sampson,"I strike quickly, being moved.",30,"(I, strike, quickly,, being, moved.)",5
